In [1]:
model_ckpt = "MCG-NJU/videomae-base" # pre-trained model from which to fine-tune
batch_size = 8 # batch size for training and evaluation

In [2]:
%env WANDB_PROJECT = final_model_
excluded_session = 3

env: WANDB_PROJECT=final_model_


In [3]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00


In [4]:
!pip install pytorchvideo evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s et

In [5]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=2e01b3111a2b6c62145f37d626ed1b5fb8f7df209fb7d2e01e8de91a81981ae8
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [6]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.8 MB/s eta 0:00:00


In [7]:
from huggingface_hub import login

login()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import wandb
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
from huggingface_hub import hf_hub_download

hf_dataset_identifier = "minoosh/IEMOCAP_videos"
filename = "IEMOCAP_videos"
file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"
)

In [10]:
!tar xf {file_path}

In [11]:
#! du -sh /content/*

In [12]:
dataset_root_path = "IEMOCAP_videos"

!find {dataset_root_path} | head -5

IEMOCAP_videos
IEMOCAP_videos/Session2
IEMOCAP_videos/Session2/neu
IEMOCAP_videos/Session2/neu/Ses02F_impro01_M000.mp4
IEMOCAP_videos/Session2/neu/Ses02M_impro05_F004.mp4


In [13]:
import pathlib

dataset_root_path = pathlib.Path(dataset_root_path)

In [14]:
session1_paths = list(dataset_root_path.glob("Session1/*/*.mp4"))
session2_paths = list(dataset_root_path.glob("Session2/*/*.mp4"))
session3_paths = list(dataset_root_path.glob("Session3/*/*.mp4"))
session4_paths = list(dataset_root_path.glob("Session4/*/*.mp4"))
session5_paths = list(dataset_root_path.glob("Session5/*/*.mp4"))

In [15]:
import os
import shutil
from sklearn.model_selection import train_test_split

def gen_train_test_structure(session, list_of_paths):

  train, test = train_test_split(list_of_paths, test_size=0.2)

  #train files
  train_dir = f'IEMOCAP_videos/Session{session}/Train/'
  os.mkdir(train_dir)
  os.mkdir(train_dir + 'neu/')
  os.mkdir(train_dir + 'sad/')
  os.mkdir(train_dir + 'hap/')
  os.mkdir(train_dir + 'ang/')

  for path in train:
    old_path = str(path )
    new_path = f'IEMOCAP_videos/Session{session}/Train/' + str(path).split(f'Session{session}/')[1]
    shutil.move(old_path, new_path)

  #test files
  test_dir = f'IEMOCAP_videos/Session{session}/Test/'
  os.mkdir(test_dir)
  os.mkdir(test_dir + 'neu/')
  os.mkdir(test_dir + 'sad/')
  os.mkdir(test_dir + 'hap/')
  os.mkdir(test_dir + 'ang/')

  for path in test:
    old_path = str(path )
    new_path = f'IEMOCAP_videos/Session{session}/Test/' + str(path).split(f'Session{session}/')[1]
    shutil.move(old_path, new_path)

  shutil.rmtree(f'IEMOCAP_videos/Session{session}/neu')
  shutil.rmtree(f'IEMOCAP_videos/Session{session}/hap')
  shutil.rmtree(f'IEMOCAP_videos/Session{session}/sad')
  shutil.rmtree(f'IEMOCAP_videos/Session{session}/ang')

In [16]:
gen_train_test_structure(1, session1_paths)
gen_train_test_structure(2, session2_paths)
#gen_train_test_structure(3, session3_paths)
gen_train_test_structure(4, session4_paths)
gen_train_test_structure(5, session5_paths)

In [17]:
class_labels = sorted({str(path).split("/")[2] for path in session1_paths})
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Unique classes: {list(label2id.keys())}.")

Unique classes: ['ang', 'hap', 'neu', 'sad'].


In [18]:
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification


feature_extractor = VideoMAEFeatureExtractor.from_pretrained(model_ckpt)
model =  VideoMAEForVideoClassification.from_pretrained(
    model_ckpt,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


Some weights of the model checkpoint at MCG-NJU/videomae-base were not used when initializing VideoMAEForVideoClassification: ['decoder.head.weight', 'decoder.decoder_layers.0.attention.output.dense.weight', 'decoder.decoder_layers.0.intermediate.dense.bias', 'decoder.decoder_layers.2.attention.output.dense.weight', 'decoder.decoder_layers.1.intermediate.dense.bias', 'decoder.decoder_layers.1.output.dense.bias', 'decoder.decoder_layers.3.output.dense.bias', 'decoder.decoder_layers.0.attention.attention.key.weight', 'decoder.norm.weight', 'decoder.decoder_layers.0.layernorm_before.bias', 'decoder.decoder_layers.2.layernorm_before.bias', 'decoder.decoder_layers.0.layernorm_before.weight', 'decoder.decoder_layers.3.layernorm_after.weight', 'decoder.decoder_layers.2.attention.attention.q_bias', 'decoder.decoder_layers.0.layernorm_after.bias', 'decoder.decoder_layers.1.attention.attention.q_bias', 'decoder.decoder_layers.0.intermediate.dense.weight', 'decoder.decoder_layers.3.attention.atte

In [19]:
import pytorchvideo.data

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [20]:
mean = feature_extractor.image_mean
std = feature_extractor.image_std
#resize_to = feature_extractor.size
resize_to = feature_extractor.size['shortest_edge']

num_frames_to_sample = model.config.num_frames
sample_rate = 4
fps = 30
clip_duration = num_frames_to_sample * sample_rate / fps

# Training dataset transformations.
train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    RandomShortSideScale(min_size=256, max_size=320),
                    RandomCrop(resize_to),
                    RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)
# Training dataset.
train_dataset1 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session1/Train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)
train_dataset2 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session2/Train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)
'''train_dataset3 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session3/Train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)'''
train_dataset4 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session4/Train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)
train_dataset5 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session5/Train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)

In [21]:
# Validation and evaluation datasets' transformations.
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    Resize((resize_to, resize_to)),
                ]
            ),
        ),
    ]
)

# Validation and evaluation datasets.
val_dataset1 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session1/Test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)
val_dataset2 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session2/Test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)
'''val_dataset3 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session3/Test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)'''
val_dataset4 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session4/Test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)
val_dataset5 = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session5/Test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

In [22]:
test_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "Session3"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

In [23]:
import torch
train_dataset = torch.utils.data.ChainDataset([train_dataset1,
                                               train_dataset2,
                                               #train_dataset3,
                                               train_dataset4,
                                               train_dataset5
                                               ])
val_dataset = torch.utils.data.ChainDataset([val_dataset1,
                                             val_dataset2,
                                             #val_dataset3,
                                             val_dataset4,
                                             val_dataset5
                                               ])

In [24]:
# We can access the `num_videos` argument to know the number of videos we have in the
# dataset.
train_dataset.num_videos = sum([train_dataset1.num_videos,
                                train_dataset2.num_videos,
                                #train_dataset3.num_videos,
                                train_dataset4.num_videos,
                                train_dataset5.num_videos])
val_dataset.num_videos = sum([val_dataset1.num_videos,
                              val_dataset2.num_videos,
                              #val_dataset3.num_videos,
                              val_dataset4.num_videos,
                              val_dataset5.num_videos])

In [25]:
from transformers import TrainingArguments, Trainer

model_name = model_ckpt.split("/")[-1]
new_model_name = f"{model_name}-finetuned-IEMOCAP_3"
num_epochs = 10

args = TrainingArguments(
    new_model_name,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    num_train_epochs=num_epochs,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    max_steps=(train_dataset.num_videos // batch_size) * num_epochs,
    report_to='wandb',
    run_name='finetune_videomae_on_IEMOCAP_3'

)

In [26]:
import evaluate

metric = evaluate.load("accuracy")

In [27]:
import numpy as np

In [28]:
# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [29]:
import torch


def collate_fn(examples):
    """The collation function to be used by `Trainer` to prepare data batches."""
    # permute to (num_frames, num_channels, height, width)
    pixel_values = torch.stack(
        [example["video"].permute(1, 0, 2, 3) for example in examples]
    )
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

Cloning https://huggingface.co/minoosh/videomae-base-finetuned-IEMOCAP_3 into local empty directory.


In [31]:
train_results = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
0,1.364800,1.366060,0.374609
1,1.318800,1.343591,0.321451
2,1.282600,1.365075,0.321451
3,1.214100,1.317015,0.375860
4,1.255500,1.301753,0.405253
5,1.242200,1.303631,0.356473
6,1.277600,1.265010,0.392120
7,1.207400,1.265450,0.397123
8,1.235400,1.254153,0.395247
9,1.086500,1.249826,0.407755


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

In [32]:
trainer.evaluate(test_dataset)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


{'eval_loss': 1.3700705766677856,
 'eval_accuracy': 0.317907444668008,
 'eval_runtime': 549.1342,
 'eval_samples_per_second': 3.62,
 'eval_steps_per_second': 0.453,
 'epoch': 9.1}

In [33]:
trainer.push_to_hub()

To https://huggingface.co/minoosh/videomae-base-finetuned-IEMOCAP_3
   7a5714f..ffabcb5  main -> main

   7a5714f..ffabcb5  main -> main

